# Create Neo4j Database

### Retrieve Required Data

In [1]:
import pandas as pd
import pickle

In [6]:
# User comedian matrix
matrix_filename = './user_comedian_matrix_OriginalPolarity.pkl'
ucm_df = pd.read_pickle(matrix_filename)
ucm_df.head()

,Ramy Youssef,Andy Woodhull,Amy Schumer,Arsenio Hall,Aziz Ansari,Bert Kreischer,Bill Burr,Chris D'Elia,Chris Rock,Dave Chappelle,...,Ronny Chieng,Roy Wood Jr.,Russell Peters,Sebastian Maniscalco,Tiffany Haddish,Tom Segura,Trevor Noah,Urzila Carlson,Vir Das,Whitney Cummings
A B,0.191429,0.000000,0.0,0.00,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0000,0.0,0.0,0.0,0.0,0.0,0.0
ADEDOTUN AKANDE,0.000000,0.875000,0.0,0.00,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.4,0.0,-0.1625,0.0,0.0,0.0,0.0,0.0,0.0
Anton Nym,0.000000,0.680990,0.0,0.00,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0000,0.0,0.0,0.0,0.0,0.0,0.0
Lydia Waweru-Morgan,0.000000,-0.237500,0.0,0.65,0.0,0.488281,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0000,0.0,0.0,0.0,0.0,0.0,0.0
Jennifer Bates,0.000000,0.671875,0.0,0.00,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0000,0.0,0.0,0.0,0.0,0.0,0.0


In [17]:
# Comedian & keywords
kw_df = pd.read_excel('keywords_noun.xlsx',index_col=0)
kw_df.head()

,Comedian,Keywords
0,amy schumer,"thats thing, shes, kind, theyre thank, woman, ..."
1,andy woodhull,"im, thats, time wife wa, home, youre, dont, da..."
2,aziz ansari,"guy, thing, im, ims, youre, time, cause, kid, ..."
3,bert kreischer,"im, wife, shes, youre, fuck, fucking, dad, wa,..."
4,bill burr,"di, im, fucking, right, youre, person fuck, la..."


In [7]:
# Users and Comedians 
users = ucm_df.index
comedians = ucm_df.columns

### Build the graph in neo4j

In [18]:
from py2neo import Graph, Node, Relationship

In [19]:
host = "localhost"
password = "0000"
graph = Graph(host=host, password=password)

In [79]:
# Delete all nodes and relationships
DELETE_QUERY = "MATCH (n) DETACH DELETE n"
_ = graph.run(DELETE_QUERY)

In [80]:
# Add all user nodes
query = graph.begin()
for user in users:
    p = Node("Person", name=user)
    query.create(p)
_ = query.commit()

In [81]:
# Add all comedian nodes
query = graph.begin()
for comedian in comedians:
    p = Node("Comedian", name=comedian)
    query.create(p)
_ = query.commit()

In [82]:
# Add all comedian property nodes

In [84]:
# Build relationships for users and comedians
for c in ucm_df.columns:
    for u in ucm_df[c].keys():  
        rating = ucm_df.loc[u, c]
        if rating != 0:
            graph.run("""
            MATCH (u:Person{name: $user})
            MATCH (c:Comedian{name: $comedian})
            MERGE(u)-[r:commented_on{rating: $rr}]->(c)""", user=u, comedian=c, rr=rating)